# CS549 Machine Learning 
# Assignment 9: Convolutional Neural Network (Part 2) -- ConvNet for image classificaion

**Total points: 10**

In this assignment, you will implement a fully functioning ConvNet model using PyTorch. You will use the model to conduct image classification on the FashionMNST dataset.

In [5]:
import numpy as np
import matplotlib.pyplot as plt
! pip install tqdm
from tqdm import tqdm

# PyTorch is needed for this assignment
# You can install it following the instructions on the official website: https://pytorch.org/get-started/locally/
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(0)
torch.use_deterministic_algorithms(True)

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

%matplotlib inline
np.random.seed(1)


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


## Load data

Load the FashionMNIST dataset provided by PyTorch. You can also change the `download` param to `False`, and copy the "data" folder used in the previous assignment to the current folder.

See <https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader> for more information.

In [6]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True, # True
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False, # False
    download=True,
    transform=ToTensor()
)

batch_size = 64

train_loader = DataLoader(training_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

## Examine data size

Now, you can examine the size of the training/test data, which is important for determining some of the parameters of your model

In [7]:
for i, (X, y) in enumerate(train_loader):
    if i > 0:
        break

print('X.shape: ', X.shape)
print('Y.shape: ', y.shape)

X.shape:  torch.Size([64, 1, 28, 28])
Y.shape:  torch.Size([64])


**Expected output**:

X.shape:  torch.Size([64, 1, 28, 28])
y.shape:  torch.Size([64])

***

## Task 1. Build the model
**8 points**

You will need to define your ConvNet model as a subclass of `torch.nn.Module`. Becuase we have already imported `torch.nn` as `nn`, we can specify the baseclass simply as `nn.Module`.

You need to override two functions in defining the class, `__init__()` and `forward()`.
- All the parameters, including the convolutional, pooling, and fully-connected layers are defined in `__init__()`. They are declared and initialized as members of the class, using the `self.` notation in Python. 
- The forward pass of the computational graph is defined in `forward()`. This function takes as input the training data, and call all operations (conv, pool, etc.) sequentially on the data. The output of a preceding operation is used as the input for the following operation. 

**Instructions:**

- Define the model so that the architecture is as follows: <br>
    Conv1 -> ReLU -> BatchNorm-> MaxPool1 -> \
    Conv2 -> ReLU -> BatchNorm-> MaxPool2 -> \
    FullyConnected -> Softmax.
  <br> in which,\
    - `conv1` has filter size $f=3$, stride $s=1$, padding $p=0$, the number of filters $n_f=6$
    - `conv2` has filter  $f=3$, stride $s=2$, padding $p=0$, the number of filters $n_f=12$;
    - all max-pool layers use filter  $f=2$ (stride $s=2$ by default).
  <br>
- *Note* that the *RELU* activation function is implemented in `forward()` rather than `__init__()`, using `F.relu()`, in which `F` is short for `torch.nn.functional` (imported at the beginning).

- The `in_features` of `self.fc` is the total number of output units after the `self.pool2` layer.
- The `out_features` of `self.fc` should match the number of classes in FashionMNIST dataset, which is 10.
- Use the following formula to compute the height and width of ouputs from conv layers.
\begin{equation}\text{Output} = (\lfloor\frac{n+2p-f}{s}\rfloor + 1)\times(\lfloor\frac{n+2p-f}{s}\rfloor + 1)\end{equation}
- For the output of model, need to use `nn.logSoftmax()`.

In [8]:
class ConvNetModel(nn.Module):
    def __init__(self, debug=False):
        super(ConvNetModel, self).__init__()
        self.debug = debug

        ### START YOUR CODE ###
        # The first convolutional layer has in_channels=1, out_channels=6, kernel_size=3, with default stride=1 and padding=0
        self.conv1 = nn.Conv2d(1, 6, kernel_size=3, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(6)
        # The first pooling layer is a maxpool with a square window of kernel_size=2 (default stride is same as kernel_size)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # The second convolutional layer
        # NOTE: Its in_channels should match the out_channels of conv1
        self.conv2 = nn.Conv2d(6, 16, kernel_size=3, stride=1, padding=0)
        self.bn2 = nn.BatchNorm2d(16)
        # The second pooling layer is maxpool with a square window of kernel_size=2
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # The fully-connected layer
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(400, 10) # Use nn.Linear, and you need to specify the correct in_features and out_features
        
        # Softmax layer
        self.output = nn.LogSoftmax(dim=1) # Use nn.LogSoftmax(), specify the dim correctly
        ### END YOUR CODE ###
        
    
    def forward(self, x):
        # Conv1 -> ReLU -> Batchnorm1-> Pool1
        x = self.pool1(self.bn1(F.relu(self.conv1(x))))
        if self.debug:
            print('output shape of pool1:', x.shape)
        
        ### START YOUR CODE ###
        # Conv2 -> ReLU -> Batchnorm2 -> Pool2
        x = self.pool2(self.bn2(F.relu(self.conv2(x))))
        if self.debug:
            print('output shape of pool2:', x.shape)
        
        # Flatten the output from the last pooling layer
        x = self.flatten(x)
        
        # Call the fully-connected layer, followed by a F.relu()
        x = F.relu(self.fc(x))
        
        # Call softmax layer
        x = self.output(x)
        ### END YOUR CODE ###
        
        return x

In [9]:
model = ConvNetModel(debug=False) # You can use debug mode to help

# Do not change the test code below
torch.manual_seed(0)
input_data = torch.randn(64, 1, 28, 28)
output = model(input_data)

print('output.size():', output.size())

output.size(): torch.Size([64, 10])


### Expected output

output.size(): torch.Size([64, 10])

***

## Task 2. Train and evaluate
**2 points**

Now you will use the functions you have implemented above to build a full model. Then you train the model on the sign language dataset.

You can refer to the previous assignment or the official documents: See <https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html> and <https://pytorch.org/docs/stable/optim.html> for more information.

In [10]:
def train_loop(dataloader, model, loss_fn, optimizer, verbose=True):
    for i, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        ### START YOUR CODE ###
        pred = model(X) # Get the prediction output from model
        loss = loss_fn(pred, torch.tensor(y)) # compute loss by calling loss_fn()
        ### END YOUR CODE ###

        # Backpropagation
        ### START YOUR CODE ###
        optimizer.zero_grad() # zero_grad()
        loss.backward() # backward()
        optimizer.step() # step()
        ### END YOUR CODE ###

        if verbose and i % 100 == 0:
            loss = loss.item()
            current_step = i * len(X)
            print(f"loss: {loss:>7f}  [{current_step:>5d}/{len(dataloader.dataset):>5d}]")

In [13]:
import torch
@torch.no_grad()
def test_loop(dataloader, model, loss_fn):
    test_loss, correct = 0, 0

    for X, y in dataloader:
        ### START YOUR CODE ###
        pred = model(X)
        loss = loss_fn(pred, torch.tensor(y))
        test_loss += loss.item()
        correct += (pred.argmax(1) == y).sum().item() # Add the number of correct prediction in the current batch to `correct`
        ### END YOUR CODE ###

    test_loss /= len(dataloader)
    ### START YOUR CODE ###
    test_acc = correct / len(dataloader.dataset)
    ### END YOUR CODE ###

    print(f"Test Error: \n Accuracy: {(100*test_acc):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Next, execute the following cell to start the training and testing loop.

**Note** that a different loss function, `nn.NLLLoss()` should be used, instead of `nn.CrossEntropyLoss()`, because we already used softmax as the output layer in the model.

In [14]:
model = ConvNetModel() # Reset the model
learning_rate = 1e-3

### START YOUR CODE ###
loss_fn = nn.NLLLoss() # Use the correct loss function
### END YOUR CODE ###

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    ### START YOUR CODE ###
    train_loop(train_loader, model, loss_fn, optimizer, verbose=False) # Call train_loop(); use verbose=False, if you want to see less information
    test_loop(test_loader, model, loss_fn) # Call test_loop()
    ### END YOUR CODE ###

print("Done!")

Epoch 1
-------------------------------


/var/folders/b6/6366288j429498w43rvsw48w0000gn/T/ipykernel_73571/3999771966.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = loss_fn(pred, torch.tensor(y)) # compute loss by calling loss_fn()
/var/folders/b6/6366288j429498w43rvsw48w0000gn/T/ipykernel_73571/1551708294.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = loss_fn(pred, torch.tensor(y))


Test Error: 
 Accuracy: 85.6%, Avg loss: 0.398487 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 87.1%, Avg loss: 0.365361 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 87.5%, Avg loss: 0.352493 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 87.7%, Avg loss: 0.345838 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 87.8%, Avg loss: 0.340217 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 88.0%, Avg loss: 0.338452 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 88.2%, Avg loss: 0.335389 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 88.2%, Avg loss: 0.333538 

Epoch 9
-------------------------------
Test Error: 
 Accuracy: 88.2%, Avg loss: 0.334388 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 88.3%, Avg loss: 0.332376 

Done!


### Expected output

You should be able to reach above 85% test accuracy.

***

## Congratulations!
Now you have successfully built a convolutional neural network model for image classification! 
Hopefully this experience of using PyTorch will help you with your final project.